In [ ]:
# import packages
import requests
import json
import pandas as pd
import pickle
import time
import os

In [ ]:
# Get acces API
url = "https://api.edamam.com/api/recipes/v2"

api_id = os.environ['APP_ID']
api_key = os.environ['APP_KEY'] 

headers = {api_id, api_key}

querystring = {"type": "public", "q":"dash", "app_id": api_id, "app_key": api_key}

response = requests.get(url, headers={"Authorization": api_key}, params=querystring)

In [ ]:
json_response = json.loads(response.text.replace('null', '"None"').replace('True','"True"').replace('False','"False"'))

In [ ]:
recipe_request = json_response

In [ ]:
still_some_data = True
next_url=url

In [ ]:
rec_dic = {}
count = 0 
df_list = []
while still_some_data:
    response = requests.get(next_url, headers={"Authorization": api_key}, params=querystring)
    try: 
      response.json()
    except:
      still_some_data = False
    try:
      recipe_request = response.json()
    except:
      print('headersize: ', print(len(next_url)))
      print(response.text) 
    time.sleep(6)
    
    try:
        print(f'Getting data for '+str(recipe_request['to']))
    except json.decoder.JSONDecodeError:
        print("There was a problem accessing the equipment data.")
        
    if recipe_request['to']==recipe_request['count']:
        still_some_data=False
    if recipe_request['to']> recipe_request['count']:
        still_some_data = False 
        
    next_url = recipe_request['_links']['next']['href']

    
    for idx, recipe in enumerate(recipe_request['hits']):
      count+=1
      if (recipe["recipe"]["mealType"][0]== 'lunch/dinner'):
        # print(recipe["recipe"]["label"])
        df_list.append([recipe["recipe"]["label"], \
            str(round(recipe['recipe']["totalNutrients"]['PROCNT']['quantity'], 2))+" g", \
            str(round(recipe['recipe']["totalNutrients"]['CHOCDF']['quantity'], 2))+" g",\
            str(round(recipe['recipe']["totalNutrients"]['FAT']['quantity'], 2))+" g"])

In [ ]:
print(df_list)
print(len(df_list))


In [ ]:
df = pd.DataFrame(df_list, columns = ['Recipe_name', 'Protein', 'Carbs', 'Fat'])

print(df)
print(len(df['Recipe_name'].unique()))

In [ ]:
# Save into CSV
df.to_csv('Dash.csv', index = False)

In [ ]:
# Or read pandas
pd.read_csv('Dash.csv')